In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import requests
import json
from datetime import datetime, date, timedelta
from pytz import timezone
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
API_key = os.getenv("aero_api")

In [5]:
# read the city dataframe to get the names of all cities we want airport data for
city_data = pd.read_csv("city_data.csv")
city_data.drop(columns = ["Unnamed: 0"], inplace = True)
city_data

,city_id,city,country,country_code,region,elevation,city_latitude,city_longitude,population
0,Q64,Berlin,Germany,DE,Berlin,NaN,52.516667,13.383333,3664088
1,Q1055,Hamburg,Germany,DE,Hamburg,NaN,53.550000,10.000000,1852478
2,Q1726,Munich,Germany,DE,Bavaria,519.0,48.134520,11.571000,1488202
3,Q365,Cologne,Germany,DE,North Rhine-Westphalia,52.0,50.942222,6.957778,1083498
4,Q1794,Frankfurt am Main,Germany,DE,Hesse,112.0,50.113611,8.679722,764104
5,Q1022,Stuttgart,Germany,DE,Baden-Württemberg,245.0,48.776111,9.177500,630305
6,Q1718,Düsseldorf,Germany,DE,North Rhine-Westphalia,38.0,51.231144,6.772381,645923
7,Q1295,Dortmund,Germany,DE,North Rhine-Westphalia,86.0,51.513889,7.465278,587696
8,Q2066,Essen,Germany,DE,North Rhine-Westphalia,116.0,51.450833,7.013056,582415
9,Q2079,Leipzig,Germany,DE,Saxony,113.0,51.340000,12.375000,597215


In [7]:
# convert the latitude column to a list to be used as input of a function call
lat = city_data["city_latitude"].to_list()
lat

[52.516666666,
 53.55,
 48.13452,
 50.942222222,
 50.113611111,
 48.776111111,
 51.231144444,
 51.513888888,
 51.450833333,
 51.34,
 48.208333333,
 47.070833333,
 48.3,
 47.80067,
 47.268333333]

In [8]:
# convert the longitude column to a list to be used as input of a function call
lon = city_data["city_longitude"].to_list()
lon

[13.383333333,
 10.0,
 11.571,
 6.957777777,
 8.679722222,
 9.1775,
 6.772380555,
 7.465277777,
 7.013055555,
 12.375,
 16.3725,
 15.438611111,
 14.283333333,
 13.04532,
 11.393333333]

In [9]:
# example call to get airports from location
url = "https://aerodatabox.p.rapidapi.com/airports/search/location/51.511142/-0.103869/km/100/16"
querystring = {"withFlightInfoOnly":"true"}
headers = {
    "X-RapidAPI-Key": API_key,
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}
response = requests.request("GET", url, headers = headers, params = querystring)
response.json()

{'items': [{'icao': 'EGLC',
   'iata': 'LCY',
   'name': 'London, London City',
   'shortName': 'City',
   'municipalityName': 'London',
   'location': {'lat': 51.5053, 'lon': 0.055277},
   'countryCode': 'GB'},
  {'icao': 'EGLL',
   'iata': 'LHR',
   'name': 'London, London Heathrow',
   'shortName': 'Heathrow',
   'municipalityName': 'London',
   'location': {'lat': 51.4706, 'lon': -0.461941},
   'countryCode': 'GB'},
  {'icao': 'EGKK',
   'iata': 'LGW',
   'name': 'London, London Gatwick',
   'shortName': 'Gatwick',
   'municipalityName': 'London',
   'location': {'lat': 51.1481, 'lon': -0.190277},
   'countryCode': 'GB'},
  {'icao': 'EGGW',
   'iata': 'LTN',
   'name': 'London, London Luton',
   'shortName': 'Luton',
   'municipalityName': 'London',
   'location': {'lat': 51.8747, 'lon': -0.368333},
   'countryCode': 'GB'},
  {'icao': 'EGSS',
   'iata': 'STN',
   'name': 'London, London Stansted',
   'shortName': 'Stansted',
   'municipalityName': 'London',
   'location': {'lat': 5

In [ ]:
# loop through the list of cities and make API calls to get airport data and concatenate them into one dataframe
airport_list = []
for i in range(len(lat)):
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat[i]}/{lon[i]}/km/100/20"
    querystring = {"withFlightInfoOnly":"true"}
    headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers = headers, params = querystring)
    airport_df = pd.json_normalize(response.json()["items"])
    airport_list.append(airport_df)
airports_df = pd.concat(airport_list, ignore_index = True)
airports_df

In [10]:
# rewrite the code above as a function so it can be reused again elsewhere
def icao_airport_code(latitude, longitude):
    airport_list = []
    # check the length of the latitude and longitude lists to make sure they are equal
    assert len(latitude) == len(longitude)
    # set the API call to get airport data within 50km of the lat and lon being input and show 10 results
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location/51.511142/-0.103869/km/50/10"
    querystring = {"withFlightInfoOnly":"true"}
    headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    test = requests.request("GET", url, headers = headers, params = querystring)
    if test.status_code >= 200 and test.status_code <= 299:
        for i in range(len(latitude)):
            url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitude[i]}/{longitude[i]}/km/50/10"
            querystring = {"withFlightInfoOnly":"true"}
            headers = {
                "X-RapidAPI-Key": API_key, 
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers = headers, params = querystring)
            airport_df = pd.json_normalize(response.json()["items"])
            airport_list.append(airport_df)
    else:
        return -1
    airports_df = pd.concat(airport_list, ignore_index = True)
    return airports_df

In [11]:
airport_data = icao_airport_code(lat, lon)
airport_data

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon,localCode
0,EDDT,TXL,"Berlin, Berlin-Tegel",-Tegel,Berlin,DE,52.55970,13.287699,NaN
1,EDDB,BER,"Berlin, Berlin Brandenburg",Brandenburg,Berlin,DE,52.35139,13.493889,NaN
2,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,53.63040,9.988229,NaN
3,EDDM,MUC,Munich,Munich,Munich,DE,48.35380,11.786100,NaN
4,EDDK,CGN,"Cologne, Cologne Bonn",Bonn,Cologne,DE,50.86590,7.142739,NaN
5,EDDL,DUS,"Duesseldorf, Düsseldorf",Düsseldorf,Duesseldorf,DE,51.28950,6.766779,NaN
6,EDDF,FRA,Frankfurt-am-Main,Frankfurt-am-Main,Frankfurt-am-Main,DE,50.02640,8.543129,NaN
7,EDDS,STR,Stuttgart,Stuttgart,Stuttgart,DE,48.68990,9.221960,NaN
8,EDDL,DUS,"Duesseldorf, Düsseldorf",Düsseldorf,Duesseldorf,DE,51.28950,6.766779,NaN
9,ETUR,BGN,Brüggen Air Base,NaN,NaN,DE,51.19970,6.132080,БЯГ


In [12]:
# remove all air bases from the result
airport_data = airport_data[~airport_data.name.str.contains("Air Base", case = False)]

In [13]:
# remove all duplicate icao
airport_data.drop_duplicates(subset = "icao", inplace = True)

In [14]:
# drop unnessary columns
airport_data.drop(columns = ["shortName", "localCode"], inplace = True)

In [15]:
# rename columns with more intuitive names and get rid of dots to avoid problems in MySQL
airport_data.rename(columns = {"name": "airport_name",
                               "municipalityName": "municipality_name",
                               "countryCode": "country_code",
                               "location.lat": "airport_latitude",
                               "location.lon": "airport_longitude"},
                    inplace = True)

In [16]:
# edit entries in the municipality_name column to match the city name for a merge operation later
airport_data.at[5, "municipality_name"] = "Düsseldorf"

In [17]:
airport_data.at[6, "municipality_name"] = "Frankfurt am Main"

In [18]:
# merge the airport data with the city data to add a city_id column to airport data
airport_data = airport_data.merge(city_data[["city_id", "city"]], how = "left", left_on = "municipality_name", right_on = "city")

In [19]:
airport_data.drop(columns = ["city"], inplace = True)

In [20]:
airport_data.reset_index(drop = True, inplace = True)

In [21]:
airport_data

,icao,iata,airport_name,municipality_name,country_code,airport_latitude,airport_longitude,city_id
0,EDDT,TXL,"Berlin, Berlin-Tegel",Berlin,DE,52.55970,13.287699,Q64
1,EDDB,BER,"Berlin, Berlin Brandenburg",Berlin,DE,52.35139,13.493889,Q64
2,EDDH,HAM,Hamburg,Hamburg,DE,53.63040,9.988229,Q1055
3,EDDM,MUC,Munich,Munich,DE,48.35380,11.786100,Q1726
4,EDDK,CGN,"Cologne, Cologne Bonn",Cologne,DE,50.86590,7.142739,Q365
5,EDDL,DUS,"Duesseldorf, Düsseldorf",Düsseldorf,DE,51.28950,6.766779,Q1718
6,EDDF,FRA,Frankfurt-am-Main,Frankfurt am Main,DE,50.02640,8.543129,Q1794
7,EDDS,STR,Stuttgart,Stuttgart,DE,48.68990,9.221960,Q1022
8,EDLW,DTM,Dortmund,Dortmund,DE,51.51830,7.612239,Q1295
9,EDDP,LEJ,"Leipzig, Leipzig Halle",Leipzig,DE,51.43240,12.241600,Q2079


In [22]:
airport_data.to_csv("airport_data.csv")

In [23]:
icao = airport_data["icao"].to_list()
len(icao)

15

In [ ]:
# arrival to EHAM airport example
today = datetime.now().astimezone(timezone("Europe/Berlin")).date()
tomorrow = (today + timedelta(days = 1))
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/{tomorrow}T10:00/{tomorrow}T22:00"
querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true",
               "withCargo":"false","withPrivate":"false","withLocation":"false"}
headers = {
    "X-RapidAPI-Key": API_key,
    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}
response = requests.request("GET", url, headers = headers, params = querystring)
pd.json_normalize(response.json()["arrivals"])

In [ ]:
# loop through the list of airport icao codes and make API calls to get flight arrival data and concatenate 
# them into one dataframe
arrival_list = []
for code in icao:
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{code}/2022-06-09T10:00/2022-06-09T22:00"
    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true",
                   "withCargo":"false","withPrivate":"false","withLocation":"false"}
    headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers = headers, params = querystring)
    arrival_df = pd.json_normalize(response.json()["arrivals"])
    arrival_list.append(arrival_df)
arrivals_df = pd.concat(arrival_list, ignore_index = True)  
arrivals_df

In [24]:
# rewrite the code above as a function
def get_flight_data(icao):
    # use the datetime function in python to get today's and tomorrow's date to be used as inputs of the API call
    today = datetime.now().astimezone(timezone("Europe/Berlin")).date()
    tomorrow = (today + timedelta(days = 1))
    arrival_list = []
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/{tomorrow}T10:00/{tomorrow}T22:00"
    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true",
                   "withCargo":"false","withPrivate":"false","withLocation":"false"}
    headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    test = requests.request("GET", url, headers = headers, params = querystring)
    if test.status_code >= 200 and test.status_code <= 299:
        for code in icao:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{code}/{tomorrow}T10:00/{tomorrow}T22:00"
            querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false",
                           "withCodeshared":"true","withCargo":"false","withPrivate":"false",
                           "withLocation":"false"}
            headers = {
                "X-RapidAPI-Key": API_key,
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers = headers, params = querystring)
            arrival_df = pd.json_normalize(response.json()["arrivals"])
            arrival_df["arrival_icao"] = code
            arrival_list.append(arrival_df)
    else:
        return -1
    arrivals_df = pd.concat(arrival_list, ignore_index = True)  
    return arrivals_df

In [25]:
flight_data = get_flight_data(icao)
flight_data.head()

,arrival_icao,number,status,codeshareStatus,isCargo,movement.airport.name,movement.scheduledTimeLocal,movement.scheduledTimeUtc,movement.terminal,movement.quality,...,airline.name,movement.airport.icao,movement.airport.iata,aircraft.reg,aircraft.modeS,callSign,movement.actualTimeLocal,movement.actualTimeUtc,movement.gate,movement.baggageBelt
0,EDDB,EW 50,Unknown,Unknown,False,Keln,2022-06-16 15:40+02:00,2022-06-16 13:40Z,1,[Basic],...,Eurowings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDDB,EW 8055,Unknown,Unknown,False,Keln,2022-06-16 20:30+02:00,2022-06-16 18:30Z,1,[Basic],...,Eurowings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EDDB,FI 528,Unknown,Unknown,False,Reykjavik,2022-06-16 13:10+02:00,2022-06-16 11:10Z,1,[Basic],...,Icelandair,BIKF,KEF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EDDB,FI 526,Unknown,Unknown,False,Reykjavik,2022-06-16 16:30+02:00,2022-06-16 14:30Z,1,[Basic],...,Icelandair,BIKF,KEF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EDDB,U2 4580,Unknown,Unknown,False,Prishtina,2022-06-16 12:50+02:00,2022-06-16 10:50Z,1,[Basic],...,easyJet,BKPR,PRN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
flight_data.drop(columns = ["codeshareStatus", "isCargo", "movement.scheduledTimeUtc", "movement.quality", 
                             "aircraft.reg", "aircraft.modeS", "callSign", "movement.actualTimeLocal",
                             "movement.actualTimeUtc", "movement.gate", "movement.baggageBelt"], inplace = True)

In [27]:
flight_data.rename(columns = {"number": "flight_number",
                              "movement.airport.icao": "departure_icao",
                              "movement.airport.iata": "departure_iata",
                              "movement.airport.name": "departure_airport",
                              "movement.scheduledTimeLocal": "scheduled_time",
                              "movement.terminal": "terminal",
                              "aircraft.model": "aircraft_model",
                              "airline.name": "airline"},
                   inplace = True)

In [28]:
flight_data

,arrival_icao,flight_number,status,departure_airport,scheduled_time,terminal,aircraft_model,airline,departure_icao,departure_iata
0,EDDB,EW 50,Unknown,Keln,2022-06-16 15:40+02:00,1,Airbus A319,Eurowings,NaN,NaN
1,EDDB,EW 8055,Unknown,Keln,2022-06-16 20:30+02:00,1,Airbus A319,Eurowings,NaN,NaN
2,EDDB,FI 528,Unknown,Reykjavik,2022-06-16 13:10+02:00,1,Boeing 757-300,Icelandair,BIKF,KEF
3,EDDB,FI 526,Unknown,Reykjavik,2022-06-16 16:30+02:00,1,Boeing 737,Icelandair,BIKF,KEF
4,EDDB,U2 4580,Unknown,Prishtina,2022-06-16 12:50+02:00,1,Airbus A320,easyJet,BKPR,PRN
...,...,...,...,...,...,...,...,...,...,...
2446,LOWI,EN 8052,Unknown,Frankfurt-am-Main,2022-06-16 13:50+02:00,NaN,Embraer 195,Air Dolomiti,EDDF,FRA
2447,LOWI,U2 8291,Unknown,London,2022-06-16 10:15+02:00,NaN,Airbus A320,easyJet,EGKK,LGW
2448,LOWI,HV 6923,Unknown,Rotterdam,2022-06-16 18:05+02:00,NaN,Boeing 737-800,Transavia,EHRD,RTM
2449,LOWI,OS 903,Unknown,Vienna,2022-06-16 10:30+02:00,NaN,Embraer 195,Austrian,LOWW,VIE


In [31]:
flight_data.dtypes

arrival_icao                                        object
flight_number                                       object
status                                              object
departure_airport                                   object
scheduled_time       datetime64[ns, pytz.FixedOffset(120)]
terminal                                            object
aircraft_model                                      object
airline                                             object
departure_icao                                      object
departure_iata                                      object
dtype: object

In [30]:
flight_data["scheduled_time"] = pd.to_datetime(flight_data["scheduled_time"])

In [32]:
flight_data.to_csv("flight_data.csv")